1. Preprocessing 

In [1]:
import pandas as pd
import os as os
import numpy as np
import xlrd as xlrd
import bertopic
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
# bigram 생성에 필요한 library
from gensim.models import Phrases
from gensim.models.phrases import Phraser
# vectorize & lda에 필요한 library
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import re
import string 
string.punctuation
from bertopic.representation import MaximalMarginalRelevance
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from sentence_transformers import SentenceTransformer
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt

2024-01-12 22:44:47.553141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
base_path = '/Users/gansujin/Documents/research/2024/homehealthcare/2024/preprocessing'
record_path = os.path.join(base_path, 'tokens.xlsx')
record = pd.read_excel(record_path) 

In [3]:
note = pd.DataFrame(record, columns = ['person_id','note_text'])

In [13]:
note = note.reset_index()

Tokenizer 

In [14]:
tokens = pd.DataFrame(note, columns = ['person_id','note_text'])

In [15]:
tokens = tokens.reset_index()

In [17]:
# stopwords for department 

unitwords = ['pgdl','ngml','cm','pgml','ul','lmin','mgdl','mmoll','mmhg','x㎕', 'pak','cc','inch','rt','lt','side','cc','rd',
               'gdl','wbc','ugl','mg','g','mgml','kg','ml', 'tab','cap', '×μl', 'medication', 'wks','opd',
               'days', 'xμl', 'mlpak', 'phx','fhx', '×μl', 'identification','cheif problem','phx and fhx']
specialty_en = ['pimd', 'gimd', 'edmd','nemd','hemd','onmd','almd','opth','indm', 'ermd','cpat']
specialty_kor = ['가정의학','감염내','갑상선내분비외','급성기일반내','내분비대사내','간이식및간담도외','정신과',
                '대장항문외','류마티스내','마취통증의학','방사선종양학','비뇨의학','산부인','비뇨기과','cc','시','ent','homd','edmd','nemd',
                '성형외','소아','소아청소년','소화기내','순환기내','신경','신경외','신장내','안과','소아외과','알레르기내','영상의학','외상외','위장관외','유방외','응급의학','응급중환자외','이비인후','내분비내과',
               '이식혈관외','재활의학','정신건강의학','정형외','종양혈액내','직업환경의학','진단검사의학','취담도외','핵의학','호흡기내','흉부외','가정의학과','감염내과','갑상선내분비외과','급성기일반내과',
               '내분비대사내과','간이식및간담도외과','대장항문외과','류마티스내과','마취통증의학과','방사선종양학과','병리과','비뇨의학과','산부인과','성형외과','소아과','소아청소년과','소화기내과','순환기내과','신경과','췌담도외과',
                 '신경외과','신장내과','안과','알레르기내과','영상의학과','외상외과','위장관외과','유방외과','유방암센터','외래','종양혈액내과','월일','유방외과','신장내과',
                '응급의학과','응급중환자외과','이비인후과','이식혈관외과','재활의학과','정신건강의학과','정형외과','혈액종양내과','종양혈액','내과','내원시','본원','종양혈액내과','직업환경의학과','진단검사의학과','취담도외과','핵의학과','호흡기내과','흉부외과','정신의학과']

mystopwords = unitwords + specialty_en + specialty_kor


In [18]:
def remove_stopwords(text):
    tokens = text.split(' ')
    meaningful = [w for w in tokens if not w in mystopwords]
    return ' '.join(meaningful)

In [19]:
# Apply the remove_stopwords function to all columns
note = note.applymap(lambda x: remove_stopwords(x) if isinstance(x, str) else x)

In [26]:
def tokenize(text):
    splits = text.split( )
    words = []
    for word in splits:
        if word not in mystopwords:
            words.append(word)
    return words

In [27]:
tokenized_list = []

for text in note['note_text']:
    tokenized_list.append(tokenize(text))

In [28]:
token_df = pd.DataFrame(tokenized_list)

In [31]:
token_df['text'] = token_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

In [32]:
# Replace multiple spaces with a single space
token_df['text'] = token_df['text'].replace(r'\s+', ' ', regex=True)

In [34]:
# Your code to tokenize and filter
tokenized_list = []
drop_mask = []

for text in token_df['text']:
    tokenized = tokenize(text)
    if len(set(tokenized)) < 3:
        drop_mask.append(True)
    else:
        drop_mask.append(False)
    tokenized_list.append(tokenized)

In [35]:
# Create a copy of the original DataFrame without rows where drop_mask is True
filtered_df = token_df[~pd.Series(drop_mask)]

# Reset the index while keeping the original index in a separate column
filtered_df.reset_index(drop=True, inplace=True)
filtered_df['original_index'] = token_df[~pd.Series(drop_mask)].index

In [37]:
filtered_list = []

for text in filtered_df['text']:
    filtered_list.append(tokenize(text))

In [38]:
clean = [tkn for tkn in filtered_list if not any(stop in tkn for stop in mystopwords)]

In [40]:
clean_df = pd.DataFrame(clean)

In [42]:
clean_df['note_text'] = clean_df.iloc[:, 0:37].fillna('').apply(lambda x: ' '.join(x.astype(str)), axis=1)

In [277]:
clean_df.sentence = clean_df.apply(lambda row: re.sub(" ,", " ", row.note_text).lower(), 1) # 문장형태로 바뀐 문장들을 sentence 열에 할당합니다.

In [279]:
sentence_list = clean_df.sentence.to_list() # 토픽모델링을 진행하기 전에, sentence열을 list 타입으로 바꾸어 sentence_list에 할당합니다.

Bertopic development flow

In [280]:
# Step 1 - Extract embeddings
sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = sentence_model.encode(sentence_list, show_progress_bar=False)

In [281]:
umap_model = UMAP(n_neighbors= 5, min_dist=0.05, n_components=5, random_state=42, metric='cosine')

In [282]:
hdbscan_model = HDBSCAN(min_cluster_size = 10, metric = 'euclidean', cluster_selection_method='eom', prediction_data=True)

In [283]:
representation_model = MaximalMarginalRelevance(diversity = 0.3)

In [284]:
ctfidf_model = ClassTfidfTransformer()

In [285]:
vectorizer = TfidfVectorizer(tokenizer = tokenize, ngram_range=(1,2), stop_words = mystopwords
                        ,max_df = 0.9 # 90% 이상 등장하는 단어 제외
                        ,min_df = 2 # 1회 등장하는 단어 제외 
                        )

In [286]:
topic_model = BERTopic(language = 'korean', umap_model=umap_model, embedding_model=sentence_model, verbose = True, representation_model = representation_model,
                          hdbscan_model = hdbscan_model, vectorizer_model = vectorizer, ctfidf_model = ctfidf_model, 
                          calculate_probabilities=True, min_topic_size = 15)

In [287]:
topics, probs = topic_model.fit_transform(sentence_list, embeddings)

2024-01-12 22:21:03,399 - BERTopic - Reduced dimensionality
2024-01-12 22:21:03,596 - BERTopic - Clustered reduced embeddings


In [288]:
topic_labels = topic_model.generate_topic_labels(nr_words = 30, separator= ", ")

Optimal topic numbers

silhouette_scores = []; 1에 가까울 수록 좋다. A good silhouette score is close to 1, and it suggests that the data points within a cluster are similar to each other and dissimilar to points in other clusters.
davies_bouldin_scores = []; 0에 가까울 수록 좋다. The Davies-Bouldin index measures the compactness and separation between clusters. A lower Davies-Bouldin index indicates better clustering.
It is the average similarity ratio of each cluster with its most similar cluster. Lower values are desirable, and 0 indicates perfectly separated clusters.

In [ ]:
num_topics_list = range(1,50)
silhouette_scores = []
davies_bouldin_scores = []
best_num_topics = None
best_silhouette_score = -1
best_davies_bouldin_index = float('inf')


# Step 1 - Extract embeddings
sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = sentence_model.encode(sentence_list, show_progress_bar=False)

# Iterate over different numbers of topics
for num_topics in num_topics_list:
    # Train the topic model
    topic_model = BERTopic(
        language = 'korean', umap_model=umap_model, embedding_model=sentence_model, verbose = True, representation_model = representation_model,
                          hdbscan_model = hdbscan_model, vectorizer_model = vectorizer, ctfidf_model = ctfidf_model, 
                          calculate_probabilities=True, min_topic_size = 5, nr_topics=num_topics)


    # Generate `X` and `labels` only for non-outlier topics (as they are technically not clusters)
    umap_embeddings = topic_model.umap_model.transform(embeddings)
    indices = [index for index, topic in enumerate(topics) if topic != -1]
    X = umap_embeddings[np.array(indices)]
    labels = [topic for index, topic in enumerate(topics) if topic != -1]


    # Calculate silhouette score 
    silhouette = silhouette_score(X, labels)
    silhouette_scores.append(silhouette)

    # Calculate Davies-Bouldin score
    davies_bouldin = davies_bouldin_score(X, labels)
    davies_bouldin_scores.append(davies_bouldin)

    # Update best scores and number of topics
    if silhouette > best_silhouette_score:
        best_silhouette_score = silhouette
        best_davies_bouldin_index = davies_bouldin
        best_num_topics = num_topics

# Print the best number of topics and corresponding scores
print("Best Number of Topics:", best_num_topics)
print("Best Silhouette Score:", best_silhouette_score)
print("Best Davies-Bouldin Index:", best_davies_bouldin_index)
# Plotting
plt.plot(num_topics_list, silhouette_scores, label='Silhouette Score')
plt.plot(num_topics_list, davies_bouldin_scores, label='Davies-Bouldin Score')
plt.xlabel('Number of Topics')
plt.ylabel('Score')
plt.title('Topic Modeling Evaluation')
plt.legend()
plt.show()

In [291]:
original_document = topic_model.get_document_info(sentence_list)

In [292]:
original_document =  pd.concat([original_document, pd.DataFrame(probs)], axis=1)

Outlier reduction
: -1 (outlier topic) 에 속해서 prob 값 0 가졌던 document 에 다른 토픽 할당

In [296]:
new_topics = topic_model.reduce_outliers(sentence_list, topics, probabilities=probs, strategy="probabilities")

In [297]:
topic_model.update_topics(sentence_list, topics=new_topics)

In [298]:
topic_labels = topic_model.generate_topic_labels(nr_words = 30, separator=", ")

In [300]:
document_info = topic_model.get_document_info(sentence_list)

In [301]:
document_info =  pd.concat([document_info, pd.DataFrame(probs)], axis=1)

Merge probabilities 

In [305]:
# Define the column to exclude from the sum
column_to_exclude = 'Document'

In [306]:
# Selecting specific columns
Topic1 = document_info[['Document',0,2,11,14,17,18,24]] #post operation

In [307]:
Topic1['Topic1'] = Topic1.sum(axis=1)

In [308]:
# Selecting specific columns
Topic2 = document_info[['Document', 1, 10,22,23,26,27]] #Nutrition

In [309]:
Topic2['Topic2'] = Topic2.sum(axis=1)

In [310]:
# Selecting specific columns
Topic3 = document_info[['Document', 3,4,7,12,13,15,16,25,30]] #adl 

In [311]:
Topic3['Topic3'] = Topic3.sum(axis=1)

In [312]:
Topic4 = document_info[['Document', 5, 6,8,9,20,28,29]] #mental

In [313]:
Topic4['Topic4'] = Topic4.sum(axis=1)

In [314]:
Topic5 = document_info[['Document', 19,21,31]] #dm

In [315]:
Topic5['Topic5'] = Topic5.sum(axis=1)

In [316]:
# Concatenate the dataframes along their columns (c-bind)
result_df = pd.concat([Topic1, Topic2, Topic3, Topic4, Topic5], axis=1)

In [317]:
result_df = result_df[['Document','Topic1','Topic2','Topic3','Topic4','Topic5']]

In [321]:
result = result_df.iloc[:,5:10] 

In [324]:
result.to_excel('result.xlsx', index=True, header=True)